In [1]:
import os
import torch
from torch import nn
import torch.nn.functional as F
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torch.utils.data import DataLoader, random_split, Dataset, Subset
# from torchtext import data
# from torchvision import transforms, models
from torchmetrics import Accuracy
import pytorch_lightning as pl
from pytorch_lightning.utilities.model_summary import ModelSummary
from pytorch_lightning.loggers import WandbLogger
from PIL import Image
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import ReduceLROnPlateau, CosineAnnealingLR
import wandb
import torchmetrics
import sys
from ray.tune.integration.pytorch_lightning import TuneReportCallback
from ray import tune
from transformers import DistilBertTokenizer

C:\Users\jaydenpark\Anaconda3\envs\nlp3\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: Could not find module 'C:\Users\jaydenpark\Anaconda3\envs\nlp3\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.
  warn(f"Failed to load image Python extension: {e}")


# Lightning moduel

In [2]:
class textClassifier(pl.LightningModule):
    def __init__(self, vocab_size=10_000, learning_rate=3e-4):
        super().__init__()
        self.save_hyperparameters()
#         self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm1 = nn.LSTM(input_size=128, hidden_size=256, batch_first=True, bidirectional=True, num_layers=2, dropout=0.3)
        self.lstm2 = nn.LSTM(512, 256, bidirectional=True, num_layers=2)
#         self.lstm3 = nn.LSTM(256, 256)
        self.relu1 = nn.Sequential(
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128,64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, 1)
        )
        self.sigmoid = nn.Sigmoid()
        self.accuracy = Accuracy(task='binary')
    def forward(self, x):
#         x = self.embedding(x)
#         x = nn.utils.rnn.pack_padded_sequence(x, text_lengths, batch_first=True)
        x, (h0, c0) = self.lstm1(x)
#         hidden = torch.cat((h0[-2,:,:], h0[-1,:,:]), dim=1)
        x, (h1, c1) = self.lstm2(x,(h0,c0))
#         x, (h2, c2) = self.lstm3(x,(h1,c1))
#         print(x, x.shape)
#         print(h0, h0.shape)
#         print(hidden.shape)
        x = self.relu1(x)
        y_hat = self.sigmoid(x)
        
        return y_hat
    
    def training_step(self, batch, batch_idx):
        x = batch['input_ids']
        y = batch['labels']
        y_hat = self(x)
        loss = F.binary_cross_entropy(y_hat, y)
        pred = y_hat.round()
        acc = self.accuracy(pred, y.to(torch.int64))
        self.log("train_loss", loss, on_epoch=True, prog_bar=True)
        self.log('train_acc', acc, on_epoch=True, prog_bar=True)
        
        return {'loss':loss, 'pred':pred}
        
    def validation_step(self, batch, batch_idx):
        x = batch['input_ids']
        y = batch['labels']
        y_hat = self(x)
        loss = F.binary_cross_entropy(y_hat, y)
        pred = y_hat.round()
        acc = self.accuracy(pred, y.to(torch.int64))
        self.log("val_loss", loss, on_epoch=True, prog_bar=True)
        self.log('val_acc', acc, on_epoch=True, prog_bar=True)
        
        return {'val_loss':loss, 'pred':pred}
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adamax(self.parameters(), lr=self.hparams.learning_rate)
        return [optimizer]    

# Dataset

In [3]:
import contractions
from bs4 import BeautifulSoup
from unidecode import unidecode
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re

class textDataset(Dataset):
    def __init__(self, data_dir):
        self.stop_words = set(stopwords.words('english'))
        self.lemmatizer = WordNetLemmatizer()
        self.tweets = pd.read_csv(data_dir)
        self.tweets['keyword'] = self.tweets['keyword'].fillna('')
        self.tweets['tweet'] = self.tweets['keyword'] + ' ' + self.tweets['text']
        self.tweets = self.text_preprocessing(self.tweets)
        self.tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
    
    def rem_urls(self, data):
        regex = re.compile(f"https?://(www\.)?(\w+)(\.\w+)(/\w*)?")
        data = re.sub(regex, "", data)
        return data
    
    def rem_emails(self, data):
        regex = re.compile("([A-Za-z0-9]+[.-_])*[A-Za-z0-9]+@[A-Za-z0-9-]+(\.[A-Z|a-z]{2,})+")
        data = re.sub(regex, "", data)
        return data
    
    def rem_mention(self, data):
        regex = re.compile('@\w+')
        data = re.sub(regex, '', data)
        return data
    
    def rem_accent(self, data):
        data = unidecode(data)
        return data
    
    def rem_unicode(self, data):
        data = data.encode("ascii", "ignore").decode()
        return data
    
    def rem_punc(self, data):
        data = re.sub(f"[{string.punctuation}]", " ", data)
        return data
    
    def clean_numbers(self, data):
        data = re.sub('[0-9]{5,}', '#'*5, data)
        data = re.sub('[0-9]{4}', '#'*4, data)
        data = re.sub('[0-9]{3}', '#'*3, data)
        data = re.sub('[0-9]{2}', '#'*2, data)
        return data
    
    
    def rem_stopwords(self, data):
        return " ".join([word for word in str(data).split() if word not in self.stop_words])
    
    def rem_extra_space(self, data):
        data = re.sub(' +', ' ', data).strip()
        return data
    
    
    def lemmatize_data(self, data):
        words = [self.lemmatizer.lemmatize(word) for word in data.split()]
        data = ' '.join(words)
        return data
    
    def text_preprocessing(self, data):
        data['tweet'] = data['tweet'].str.lower()
        data['tweet'] = data['tweet'].apply(contractions.fix)
        data['tweet'] = data['tweet'].apply(self.rem_urls)
        data['tweet'] = data['tweet'].apply(self.rem_emails)
        data['tweet'] = data['tweet'].apply(lambda x: BeautifulSoup(x).get_text())
        data['tweet'] = data['tweet'].apply(self.rem_mention)
        data['tweet'] = data['tweet'].str.replace(':\(', 'sadness ')
        data['tweet'] = data['tweet'].str.replace(r':\)[$|\s]*', 'happiness ')
        data['tweet'] = data['tweet'].str.replace(r'\;\)[$|\s]*', 'happiness ')
        data['tweet'] = data['tweet'].apply(self.rem_accent)
        data['tweet'] = data['tweet'].apply(self.rem_unicode)
        data['tweet'] = data['tweet'].apply(self.rem_punc)
        data['tweet'] = data['tweet'].apply(self.clean_numbers)
        data['tweet'] = data['tweet'].apply(self.rem_stopwords)
        data['tweet'] = data['tweet'].apply(self.rem_extra_space)
        data['tweet'] = data['tweet'].apply(self.lemmatize_data)
        return data
        
    def __len__(self):
        return len(self.tweets)
    
    def __getitem__(self, idx):
        text = self.tweets.iloc[idx, -1]
        encoding = self.tokenizer.batch_encode_plus([text], return_tensors="pt", max_length=128, pad_to_max_length=True)
        input_ids = encoding['input_ids'].flatten().to(torch.float32)
        attention_mask = encoding['attention_mask'].flatten()
        label = [self.tweets.iloc[idx, -2]]
        label = torch.tensor(label, dtype=torch.float32)
        
        return {'text':text, 'input_ids':input_ids, 'attention_mask':attention_mask, 'labels':label}

In [4]:
def train_val_dataset(dataset, train_split=0.75, val_split=0.25):
    train_idx, val_idx = train_test_split(list(range(len(dataset))), train_size=train_split, test_size=val_split)
    datasets = {'train':Subset(dataset, train_idx), 'val':Subset(dataset, val_idx)}
    return datasets

In [5]:
class textDatamodule(pl.LightningDataModule):
    def __init__(self, batch_size):
        super().__init__()
        self.batch_size = batch_size
        self.tokenizer = None
    def setup(self, stage=None):
        data = textDataset("./nlp-getting-started/train.csv")
        self.tokenizer = data.tokenizer
        data = train_val_dataset(data)
        self.train_dataset = data['train']
        self.val_dataset = data['val']
    
    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True)
    
    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=self.batch_size)    

In [6]:
checkpoint_callback = pl.callbacks.ModelCheckpoint(
    dirpath='./check_point/',
    filename='{epoch}-{train_loss:.4f}-{val_loss:.4f}',
    monitor='val_loss',
    mode='min',
    save_top_k=2
)

In [7]:
wandb.init(
      mode='disabled',
      # Set the project where this run will be logged
      project="histopathologic-cancer-classifier", 
      # We pass a run name (otherwise it’ll be randomly assigned, like sunshine-lollypop-10)
      name=f"Test1", 
      # Track hyperparameters and run metadata
      config={
      "learning_rate": 0.001,
      "data_size": 1,
      "batch_size":64,
      })
wandb_logger = WandbLogger()

C:\Users\jaydenpark\Anaconda3\envs\nlp3\lib\site-packages\pytorch_lightning\loggers\wandb.py:347: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(


In [8]:
trainer = pl.Trainer(accelerator='cpu',
                    devices=1,
                    max_epochs=1000,
                    logger=wandb_logger,
                    callbacks=[checkpoint_callback]
                    )

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [9]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jaydenpark\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\jaydenpark\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [10]:
dm = textDatamodule(batch_size=wandb.config['batch_size'])
dm.setup()

C:\Users\jaydenpark\Anaconda3\envs\nlp3\lib\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(
C:\Users\jaydenpark\AppData\Local\Temp\2\ipykernel_22312\1428245944.py:74: FutureWarning: The default value of regex will change from True to False in a future version.
  data['tweet'] = data['tweet'].str.replace(':\(', 'sadness ')
C:\Users\jaydenpark\AppData\Local\Temp\2\ipykernel_22312\1428245944.py:75: FutureWarning: The default value of regex will change from True to False in a future version.
  data['tweet'] = data['tweet'].str.replace(r':\)[$|\s]*', 'happiness ')
C:\Users\jaydenpark\AppData\Local\Temp\2\ipykernel_22312\1428245944.py:76: FutureWarning: The default value of regex will change from True to False in a future version.
  data['tweet'] = data['tweet'].str.replace(r'\;\)[$|\s]*', 'happiness ')


In [11]:
text_encoder = textClassifier(vocab_size=dm.tokenizer.vocab_size)

In [ ]:
 trainer.fit(text_encoder, dm)

C:\Users\jaydenpark\Anaconda3\envs\nlp3\lib\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(
C:\Users\jaydenpark\AppData\Local\Temp\2\ipykernel_22312\1428245944.py:74: FutureWarning: The default value of regex will change from True to False in a future version.
  data['tweet'] = data['tweet'].str.replace(':\(', 'sadness ')
C:\Users\jaydenpark\AppData\Local\Temp\2\ipykernel_22312\1428245944.py:75: FutureWarning: The default value of regex will change from True to False in a future version.
  data['tweet'] = data['tweet'].str.replace(r':\)[$|\s]*', 'happiness ')
C:\Users\jaydenpark\AppData\Local\Temp\2\ipykernel_22312\1428245944.py:76: FutureWarning: The default value of regex will change from True to False in a future version.
  data['tweet'] = data['tweet'].str.replace(r'\;\)[$|\s]*', 'happiness ')
C:\Users\jaydenpark\Anaconda3\

Sanity Checking: 0it [00:00, ?it/s]

C:\Users\jaydenpark\Anaconda3\envs\nlp3\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:240: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 24 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
C:\Users\jaydenpark\Anaconda3\envs\nlp3\lib\site-packages\transformers\tokenization_utils_base.py:2285: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]